# Next
* add a send on break function: inserts 'customer' with req. call time 20 at a specified point
* Extract out queue size at all times
* cut down on frame build process time

In [3]:
# Custom style
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [161]:
import pandas as pd
import itertools
import numpy as np

In [ ]:
# import os
os.system("git add . && git commit -m 'expanding calc base' && git push")

In [ ]:
num_customerz = 24*60*30 # 10 per minute
time_periodz = 24*60*60
num_repz = 50

In [ ]:
%%time
# Lists based on the number of customers - each element represents one customer
call_in_time_list =  [np.random.choice(range(1,time_periodz)) for i in range(num_customerz)]
call_in_time_list.sort() # We'll be iterating through this list, so it should be in order
call_duration_list = [np.random.choice([1,2,3,4,5]*60) for i in range(num_customerz)] # 1 to 5 minute call times
wait_times_list = [] # This doesn't need to be done the same was as above: handled one at a time in loop

# Each element contains information about a different representative
# Next available: at all times, this represents the next time each rep will be available
next_available = [0 for x in range(num_repz)]
rep_busy = [[] for x in range(num_repz)]
rep_customers = [[] for x in range(num_repz)]

# 
for time_entry in range(len(call_in_time_list)):
    #Find representative with next availability, i.e. the minimum expected time to re-enter the queue
    index_of_prev_min = next_available.index(min(next_available))
    # for this customer, they will get off the phone at next availble time index + call time
    end_time = call_duration_list[time_entry] + max(min(next_available), call_in_time_list[time_entry])
    # update the best (least busy) rep to take this call
    next_available[index_of_prev_min] = end_time
    # for the duration of the call, the rep is busy. record a list of only those busy times
    rep_busy[index_of_prev_min].append([x for x in range(end_time - call_duration_list[time_entry], end_time)])
    # The difference between What happened (next avail + call time) and What should have happened (call in time + call time) is the wait time. 
    wait_times_list.append( end_time - (call_duration_list[time_entry] + call_in_time_list[time_entry]))
    # assign customer to rep
    rep_customers[index_of_prev_min].append(time_entry)

In [ ]:
%%time
#45 ms before this point w/ standard config
# This cell wants to grow in time quite rapidly with number of customers
df_customers = pd.DataFrame([wait_times_list, call_in_time_list, call_duration_list], index=['wait_time', 'call_in', 'call_duration']).T
df_reps = pd.DataFrame([rep_busy, rep_customers], index = ['busy_times', 'assigned_customers'])

_max_time_periods = time_periodz + max(call_duration_list)
df_txn_log = pd.DataFrame(index=[x for x in range (_max_time_periods)], columns= ['rep_' + str(i) for i in range(1, num_repz+1)])
for i in range(len(df_txn_log.columns)):
    df_txn_log.iloc[:, i]  = [1 if x in list(itertools.chain.from_iterable(df_reps.at['busy_times', i])) else 0 for x in range(_max_time_periods)]
#print(next_available)
#print(wait_times_list)
#print(rep_busy)
# Calculate q size?

In [ ]:
# deque for q stuff

In [ ]:
# import seaborn as sns
# sns.heatmap(df_txn_log.values)

In [ ]:
df_customers.wait_time.describe()

# Gut check

In [ ]:
df_txn_log.sum().sum() / sum(call_duration_list) 

In [ ]:
sum(call_duration_list) == df_txn_log.sum().sum()

In [ ]:
sns.heatmap(pd.DataFrame(df_txn_log.sum().T) / df_txn_log.shape[0])

In [ ]:
max((["{0:.2f}%".format(x) for x in (100* df_txn_log.sum().T / df_txn_log.shape[0])]))

In [ ]:
(100* df_txn_log.sum().T / df_txn_log.shape[0]).mean()

# num customers = num periods = 25*60*10 with reps == 
# 25: 12 %
# 15: 20 %, max 21.08%
# 10: 29.9% max 31.64
# 5: 59.88% max 63.65

In [ ]:
sum(call_duration_list)